In [1]:
import os
import json
import glob
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objs as go

from PIL import Image, ImageOps
from skimage import io
from skimage.color import rgba2rgb, rgb2xyz
from tqdm import tqdm
from dataclasses import dataclass
from math import floor, ceil
import random

# Train data generation
import collections
import csv
from pathlib import Path
from typing import List, Tuple, Any

import time
import re
from sklearn import preprocessing
import lightgbm as lgb

import multiprocessing
from multiprocessing import Pool, Manager

import pickle
import math
import gc
import psutil
from collections import Counter

pd.set_option("display.max_columns", 100)

In [2]:
# Settings and altering components for GCP

# path settings
root_path = "../input/indoor-location-navigation/"
# root_path = "../jupyter/input/"
train_paths = glob.glob(root_path + "train" + "/*/*/*")
test_paths = glob.glob(root_path + "test" + "/*")
metafiles = glob.glob(root_path + "metadata" + "/*")

# function imports using github repo in kaggle kernels
# https://www.kaggle.com/getting-started/71642
!cp -r ../input/indoorlocationcompetition20master/indoor-location-competition-20-master/* ./
from io_f import read_data_file
from compute_f import compute_step_positions, compute_steps, \
compute_headings, compute_stride_length, compute_step_heading, compute_rel_positions, split_ts_seq

# import for gcp settings
# import compute_f
# import io_f
# import visualize_f
# import main
# from io_f import read_data_file
# from compute_f import compute_step_positions, compute_steps, \
# compute_headings, compute_stride_length, compute_step_heading, compute_rel_positions, split_ts_seq

In [ ]:
# # Make directory for saving files
# !mkdir train
# !mkdir test

In [ ]:
# !ls ./train

In [15]:
# filter milisecond setting 
# IMU_CUT = 250
IMU_CUTS = [1000, 2000, 5000]
# WPS_CUT = 5000

# train number setting
# TRAIN_NUM = len(train_paths)
# TRAIN_NUM = round(len(train_paths) / 2)
TRAIN_NUM = 10

# floor translation
FLOOR_MAP = {"B3":-3,"B2":-2,"B1":-1,"F1":0,"1F":0,"F2":1,"2F":1,"F3":2,"3F":2,"F4":3,"4F":3,
             "F5":4,"5F":4,"F6":5,"6F":5,"F7":6,"7F":6,"F8":7,"8F": 7,"F9":8,"9F":8,"F10":9,
             "B":0,"BF":1,"BM":2, "G":0, "M":0, "P1":0,"P2":1, "LG2":-2,"LG1":-1,"LG":0,"LM":0,
             "L1":1,"L2":2,"L3":3,"L4":4,"L5":5,"L6":6,"L7":7,"L8":8,"L9":9,"L10":10,"L11":11}

# Columns to shift to the beginning of df
SHIFT_COLS = ["rel_y", "rel_x", "rel_diff", \
              "magn_u_z_avg", "magn_u_y_avg", "magn_u_x_avg", \
              "gyro_z_avg", "gyro_y_avg", "gyro_x_avg", \
              "ahrs_z_avg", "ahrs_y_avg", "ahrs_x_avg",  \
              "magn_st", "magn_z_avg", "magn_y_avg", "magn_x_avg", \
              "acce_z_avg", "acce_y_avg", "acce_x_avg", \
              "site_id", "file_id", "floor_int", "floor", \
              "y", "x", "wps_diff", "wifi_ts"]

SHIFT_COLS_TEST = ["rel_y", "rel_x", "rel_diff", \
                   "magn_u_z_avg", "magn_u_y_avg", "magn_u_x_avg", \
                   "gyro_z_avg", "gyro_y_avg", "gyro_x_avg", \
                   "ahrs_z_avg", "ahrs_y_avg", "ahrs_x_avg",  \
                   "magn_st", "magn_z_avg", "magn_y_avg", "magn_x_avg", \
                   "acce_z_avg", "acce_y_avg", "acce_x_avg", \
                   "site_id", "file_id", "floor_int", "floor", \
                   "y", "x", "wps_diff", "wifi_ts", "site_path_timestamp"]

INT_COLS = ["wifi_ts"]
CAT_COLS = ["file_id", "site_id", "floor"]

In [4]:
# Preprocess
print("No. Files in Train: {:,}".format(len(train_paths)), "\n" +
      "No. Files in Test: {:,}".format(len(test_paths)), "\n" +
      "No. of metadata files: {:,}".format(len(metafiles)))

# Reading in 1 file
def pick_example(max_range, paths):
    ex = random.randint(0, max_range)
    example_path = paths[ex]
    path = f"{example_path}"
    paths = path.split("/")
    site = paths[4]
    floorNo = paths[5]
    floor_plan_filename = f"{root_path}metadata/{site}/{floorNo}/floor_image.png"
    json_plan_filename = f"{root_path}metadata/{site}/{floorNo}/floor_info.json"
    with open(json_plan_filename) as json_file:
        json_data = json.load(json_file)
    width_meter = json_data["map_info"]["width"]
    height_meter = json_data["map_info"]["height"]
    return path, site, floorNo, floor_plan_filename, json_plan_filename, width_meter, height_meter

path, site, floorNo, floor_plan_filename, \
json_plan_filename, width_meter, height_meter = pick_example(len(train_paths), train_paths)
print("example path: ", path)
print("site: ", site)
print("floorNo: ", floorNo)
print("floor_plan_filename: ", floor_plan_filename)
print("json_plan_filename: ", json_plan_filename)
print("width: {}, height: {} ".format(width_meter, height_meter))

with open(path) as p:
    lines = p.readlines()
print("No. Lines in 1 example: {:,}". format(len(lines)))

No. Files in Train: 26,925 
No. Files in Test: 626 
No. of metadata files: 204
example path:  ../input/indoor-location-navigation/train/5d2709a003f801723c3251bf/1F/5dbbb629d749ad00067c73ef.txt
site:  5d2709a003f801723c3251bf
floorNo:  1F
floor_plan_filename:  ../input/indoor-location-navigation/metadata/5d2709a003f801723c3251bf/1F/floor_image.png
json_plan_filename:  ../input/indoor-location-navigation/metadata/5d2709a003f801723c3251bf/1F/floor_info.json
width: 188.96960819209474, height: 98.62907890027277 
No. Lines in 1 example: 5,753


In [5]:
# for line in lines:
#     print(line)

In [6]:
# Redefine the data extraction class

from dataclasses import dataclass

@dataclass
class ReadData:
    acce: np.ndarray
    acce_uncali: np.ndarray
    gyro: np.ndarray
    gyro_uncali: np.ndarray
    magn: np.ndarray
    magn_uncali: np.ndarray
    ahrs: np.ndarray
    wifi: np.ndarray
    ibeacon: np.ndarray
    waypoint: np.ndarray


def read_data_file_ed(data_filename):
    acce = []
    acce_uncali = []
    gyro = []
    gyro_uncali = []
    magn = []
    magn_uncali = []
    ahrs = []
    wifi = []
    ibeacon = []
    waypoint = []

    with open(data_filename, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    for line_data in lines:
        line_data = line_data.strip()
        if not line_data or line_data[0] == '#':
            continue

        line_data = line_data.split('\t')

        if line_data[1] == 'TYPE_ACCELEROMETER':
            acce.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_ACCELEROMETER_UNCALIBRATED':
            acce_uncali.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_GYROSCOPE':
            gyro.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_GYROSCOPE_UNCALIBRATED':
            gyro_uncali.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_MAGNETIC_FIELD':
            magn.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_MAGNETIC_FIELD_UNCALIBRATED':
            magn_uncali.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_ROTATION_VECTOR':
            ahrs.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_WIFI':
            sys_ts = line_data[0]
            ssid = line_data[2]
            bssid = line_data[3]
            rssi = line_data[4]
            lastseen_ts = line_data[6]
            wifi_data = [sys_ts, ssid, bssid, '_'.join([ssid, bssid]), rssi, lastseen_ts]
            wifi.append(wifi_data)
            continue

        if line_data[1] == 'TYPE_BEACON':
            ts = line_data[0]
            uuid = line_data[2]
            major = line_data[3]
            minor = line_data[4]
            txpower = line_data[5]
            rssi = line_data[6]
            distance = line_data[7]
            mac_address = line_data[-2]
            beacon_ts = line_data[-1]
            ibeacon_data = [ts, '_'.join([uuid, major, minor]), txpower, rssi, distance, mac_address, beacon_ts]
            ibeacon.append(ibeacon_data)
            continue

        if line_data[1] == 'TYPE_WAYPOINT':
            waypoint.append([int(line_data[0]), float(line_data[2]), float(line_data[3])])

    acce = np.array(acce)
    acce_uncali = np.array(acce_uncali)
    gyro = np.array(gyro)
    gyro_uncali = np.array(gyro_uncali)
    magn = np.array(magn)
    magn_uncali = np.array(magn_uncali)
    ahrs = np.array(ahrs)
    wifi = np.array(wifi)
    ibeacon = np.array(ibeacon)
    waypoint = np.array(waypoint)

    return ReadData(acce, acce_uncali, gyro, gyro_uncali, magn, magn_uncali, ahrs, wifi, ibeacon, waypoint)

In [7]:
# Find out how many wps datapoints and wifi datapoints one floor has
train_path_floor = glob.glob(root_path + "train" + "/*/*/")
# train_paths = glob.glob(root_path + "train" + "/*/*/*")
ex = random.randint(0, 6)
print(train_path_floor[ex])
print("no. of files of that floor: ", len(os.listdir(train_path_floor[ex])))
count = 0
for f in os.listdir(train_path_floor[ex]):
    file_path = train_path_floor[ex] + f
    data = read_data_file_ed(file_path)
    count += len(data.waypoint)
    
print(count)

../input/indoor-location-navigation/train/5cdbc652853bc856e89a8694/F1/
no. of files of that floor:  49
295


In [8]:
# path, site, floorNo, floor_plan_filename, json_plan_filename, width_meter, height_meter = pick_example(len(train_paths), train_paths)
# show_site_png(root_path, site=site)

In [9]:
# Feature candidate
# You can't get the waypoint in test, so use acce and ahrs data to calculate relative positions
def calc_rel_positions(acce_datas, ahrs_datas):
    step_timestamps, step_indexs, step_acce_max_mins = compute_steps(acce_datas)
    headings = compute_headings(ahrs_datas)
    stride_lengths = compute_stride_length(step_acce_max_mins)
    step_headings = compute_step_heading(step_timestamps, headings)
    rel_positions = compute_rel_positions(stride_lengths, step_headings)
    # only use del if we don't need timestamps
    # rel_positions_del = np.delete(rel_positions, 0, 1)
    return rel_positions

# Feature candidate
# Modify extract_magnetic_strength from github for one magnetic data point
def extract_one_magn_strength(magn_datas):
    d = np.array(magn_datas)
    return np.mean(np.sqrt(np.sum(d ** 2, axis=0)))

In [10]:
# path, site, floorNo, floor_plan_filename, \
# json_plan_filename, width_meter, height_meter = pick_example(len(train_paths), train_paths)

In [11]:
# Common methods
def extract_imu_rep(imu_data, wifi_ts, imu_cut):
    imu_ts = imu_data[:, 0].astype(int)
    diff_list = []
    for ts in imu_ts:
        diff = abs(int(wifi_ts) - ts)
        diff_list.append(diff)
    # diff_idx = np.argmin(diff_list)
    # acce_diff_range = [(i,a) for i, a in enumerate(diff_list) if a < cut_line] # uncomment if we need to check acce_diff
    imu_diff_range = [i for i, a in enumerate(diff_list) if a < imu_cut]
    imu_filtered = imu_data[imu_diff_range]
    if imu_filtered.shape[0] == 0:
        print("no imu")
        imu_avg_x = np.nan
        imu_avg_y = np.nan
        imu_avg_z = np.nan
    else:
        imu_avg_x = imu_filtered[:, 1].mean()
        imu_avg_y = imu_filtered[:, 2].mean()
        imu_avg_z = imu_filtered[:, 3].mean()
        #print(imu_avg_x, imu_avg_y, imu_avg_z)
    return imu_avg_x, imu_avg_y, imu_avg_z

def shift_columns(cols, df):
    for col in cols:
        df_cols = list(df.columns)
        df_cols.insert(0, df_cols.pop(df_cols.index(col)))
        df = df[df_cols]
    return df

# convert data types of certain columns
def convert_dtypes(df, col_list, dtype):
    for col in col_list:
        df[col] = df[col].astype(dtype)

---
## Train generator
---

In [32]:
# Train specific methods
def extract_nearest_wps(wps_data, wifi_ts):
    wps_ts = wps_data[:, 0].astype(int)
    diff_list = []
    for ts in wps_ts:
        diff = abs(int(wifi_ts) - ts)
        diff_list.append(diff)
    diff_idx = np.argmin(diff_list)
    return diff_list[diff_idx], wps_data[diff_idx]

def extract_train_path(path):
    try:
        ex_path = f"{path}"
        ex_paths = ex_path.split("/")
        site_id = ex_paths[4]
        floor = ex_paths[5]
        f = FLOOR_MAP[floor]
        file_id = ex_paths[6].split(".")[0]
        return site_id, file_id, f, floor
    except:
        print("extract_path error")

def make_wifi_df_train(path):
    # First path
    datas = read_data_file_ed(path)
    
    # Put IMU data in dict for later iteration
    imu_dict = {}
    imu_dict["acce"] = datas.acce
    imu_dict["magn"] = datas.magn
    imu_dict["ahrs"] = datas.ahrs
    imu_dict["gyro"] = datas.gyro
    # acce_uncali = datas.acce_uncali
    imu_dict["magn_uncali"] = datas.magn_uncali # Only use magn for uncalibrated data, as it seems more important in initial modeling result
    # gyro_uncali = datas.gyro_uncali
    
    # Leave the non-imu data as they are
    wifi_datas = datas.wifi
    # ibeacon_datas = datas.ibeacon # ibeacon to be used only for test data
    wps = datas.waypoint
    rel_positions = calc_rel_positions(imu_dict["acce"], imu_dict["ahrs"])
    
    # print("wifi unique ts len: ", len(set(wifi_datas[:, 0])))

    # Make wifi df with wifi_ts
    dfs = []
    df = pd.DataFrame(wifi_datas[:,[0,2,4]])
    for wifi_ts, g in df.groupby(0):
        g = g.drop_duplicates(subset=1)
        tmp = g.iloc[:,1:]
        feat = tmp.set_index(1).T
        feat["wifi_ts"] = wifi_ts

        # get closest wps
        closest_wps = extract_nearest_wps(wps, wifi_ts)
        feat["wps_diff"] = closest_wps[0]
        feat["x"] = closest_wps[1][1]
        feat["y"] = closest_wps[1][2]

        # get floor and other path data
        site_id, file_id, f, floor = extract_train_path(path)
        feat["floor"] = floor
        feat["floor_int"] = f
        feat["file_id"] = file_id
        feat["site_id"] = site_id

        # Loop over IMU_CUTS
        for key, imu in imu_dict.items():
            for imu_cut in IMU_CUTS:
                imu_avgs = extract_imu_rep(imu, closest_wps[1][0], imu_cut)
                feat[f"{key}_{imu_cut}_x_avg"] = imu_avgs[0]
                feat[f"{key}_{imu_cut}_y_avg"] = imu_avgs[1]
                feat[f"{key}_{imu_cut}_z_avg"] = imu_avgs[2]
                if key == "magn":
                    feat[f"{key}_{imu_cut}_st_avg"] = extract_one_magn_strength(imu_avgs)
        
        # get closest relative positions that was worked out with acce and ahrs data
        rel_pos = extract_nearest_wps(rel_positions, wifi_ts)
        feat["rel_diff"] = rel_pos[0]
        feat["rel_x"] = rel_pos[1][1]
        feat["rel_y"] = rel_pos[1][2]
        
        dfs.append(feat)
    
    return dfs


def make_train_df(paths_df, site_list):
    for site in site_list:
        df = paths_df[paths_df["site_id"] == site]
        paths = df["path"].unique()
        # get top bssids for site
        dfs_all = pool.map(make_wifi_df_train, tqdm(paths))
        dfs_unpack = [row for df in dfs_all for row in df]
        wifi_df = pd.concat(dfs_unpack)
        all_cols = wifi_df.columns
        cols = [col for col in all_cols if len(col) < 17]
        cols = reversed(cols)
        wifi_df = shift_columns(cols, wifi_df)
        # display(wifi_df.head())
        wifi_df = wifi_df.fillna(-999)
        convert_dtypes(wifi_df, tqdm(INT_COLS), int)
        convert_dtypes(wifi_df, tqdm(CAT_COLS), "category")
        # display(wifi_df.head())
        # wifi_df.to_csv(f"./train/{site}_train.csv", index=False)
        del wifi_df

In [13]:
# train_path filtering

def extract_path_for_grouplist(path):
    ex_path = f"{path}"
    ex_paths = ex_path.split("/")
    site_id = ex_paths[4]
    file_id = ex_paths[6].split(".")[0]
    return [path, site_id, file_id]

# create pathlist to be used by 2 types of paths list
path_list = [extract_path_for_grouplist(item) for item in train_paths]
df_paths = pd.DataFrame(path_list, columns=["path", "site_id", "file_id"])
site_id_path_list = df_paths["site_id"].unique()

# grouped_paths_list -> It takes 3 records from every site_id
grouped_paths_df = df_paths.groupby("site_id").sample(n=3)
grouped_paths_list = list(grouped_paths_df["path"].unique())
display(grouped_paths_df.head())
print(len(df_paths))

,path,site_id,file_id
10528,../input/indoor-location-navigation/train/5a05...,5a0546857ecc773753327266,5e15730ca280850006f3d007
11019,../input/indoor-location-navigation/train/5a05...,5a0546857ecc773753327266,5dca83805e083300061bcbbd
10694,../input/indoor-location-navigation/train/5a05...,5a0546857ecc773753327266,5e15a2c6f4c3420006d521cf
25386,../input/indoor-location-navigation/train/5c3c...,5c3c44b80379370013e0fd2b,5d0761f64cae4f000a2db584
25453,../input/indoor-location-navigation/train/5c3c...,5c3c44b80379370013e0fd2b,5d077e134cae4f000a2db84b


26925


In [34]:
start = time.time()
num_cores = multiprocessing.cpu_count()
pool = Pool(1)

# # Checking purposes
# # 9 paths, 4 cores -> 28.5 sec
# # 9 paths, 1 core  -> 42.0 sec
grouped_paths_df = grouped_paths_df.iloc[:9,:]
# grouped_paths_df = grouped_paths_df.sample(n=100)
train_sites_list = grouped_paths_df["site_id"].unique()
make_train_df(grouped_paths_df, train_sites_list)

# REAL training
# train_sites_list = df_paths["site_id"].unique()
# make_train_df(df_paths, train_sites_list)

print("time to extract data: ", time.time() - start)
pool.close()

100%|██████████| 3/3 [00:00<00:00, 818.40it/s]


1,wifi_ts,wps_diff,x,y,floor,floor_int,file_id,site_id,acce_1000_x_avg,acce_1000_y_avg,acce_1000_z_avg,acce_2000_x_avg,acce_2000_y_avg,acce_2000_z_avg,acce_5000_x_avg,acce_5000_y_avg,acce_5000_z_avg,magn_1000_x_avg,magn_1000_y_avg,magn_1000_z_avg,magn_1000_st_avg,magn_2000_x_avg,magn_2000_y_avg,magn_2000_z_avg,magn_2000_st_avg,magn_5000_x_avg,magn_5000_y_avg,magn_5000_z_avg,magn_5000_st_avg,ahrs_1000_x_avg,ahrs_1000_y_avg,ahrs_1000_z_avg,ahrs_2000_x_avg,ahrs_2000_y_avg,ahrs_2000_z_avg,ahrs_5000_x_avg,ahrs_5000_y_avg,ahrs_5000_z_avg,gyro_1000_x_avg,gyro_1000_y_avg,gyro_1000_z_avg,gyro_2000_x_avg,gyro_2000_y_avg,gyro_2000_z_avg,gyro_5000_x_avg,gyro_5000_y_avg,gyro_5000_z_avg,rel_diff,rel_x,rel_y,...,87b3224d2939d5e4e116776e50e6edf0f3f7beaf,edfec57e7fdec0f9decbe1e65d7e04539d4e68ba,36c875bc5abf26f3324b443b87255971d16116e6,e45464bcb96fe0ba1573d057553be653e464a022,f427cfddb00aafb1d9b96eb02aeecb7afd8eb0c0,6ae5a4d4f1aceb46c9b1f86f188b635ba706803f,b2589052f0de9a06aa57350ae44782e9a22bb9dd,e7d431b2031b1f47e69f73d608d11422838b72fa,5de4cbc7a398c0739031a471fe8a92d1cee23e03,d3b97faf3bd3d708ce5a438e780f55703a8a319b,8985e3c3754e5e2507071da4b53bd14c3eebaf4f,93dcb3a20e28d2bae3583cb1df106ecea3f755b0,d5dad1fcdae9e773ede884b3b4d781d5ee1ec90e,90431862956e7ac5095ce42fb7410a72d4581369,20a605f262a4c6c433a5ea6a5838990fd6e79199,a43425de6069b069e47428dc1914399cbb560afc,a107122e6a880623705d91b8bd5f71ea3868ce42,b6af8823fc4ae3ce514ee8de422389a993d413a7,19018dd66fa2c658a8b9937f2e8062a508cacc97,b5a7a6e68677c0eafca4d11cde2b41a7f68f8a21,fa993beb9ebe0c121f0150da8182c380e028fd52,bf1bfc94a6a022ee0c63ba5c164122d7a5839dfc,c254e10b286149f153381ad9c19b0fba9e39f9e4,13452c4523892bfea71e55380f301ed7fef37215,9814a3bcf1be199c11849db9640caa663a534a2b,fdfad99711e053cd7dca1057f8dfb80b2f275728,dbf834c29e836ceb11ed67250ff022e0574ae151,4e3000bacc68d040917f0161eb42e9eeed321be9,c85e90020272ea32feae92d3f50e87f861d2e43a,568f48392e897ae028e0a013c500ab3c8d3d30f8,4a8e16eed78042df1d5f4dd2b7c7488d19999aef,2051a84cae057ecd3125bb1b3396d5414baf6aae,d5e224d8c5183956d0fba256aea2d9e21803f441,62ceea2a1369d0a07b88b57829bfd29f75e8aca1,c90fa8e289f6d3c4772b4e9d5bafb3034ab119db,827b3dd8d0c3d3bd2c77643d5a173cd63ba93c0a,707a33db19f2064df5f4e9f03dbb439aff81fbea,0e2250a84319bd6fb557786aa36b3c329f9b4fd1,0328276961f014a552c7c7836123c5b3707bd51c,6f6d9358d63072a7343657da2584471a026ef222,aa06bb847e57b590229d854f60e7965881c39bc9,12911a64fecf13f2e9fb0aaed554621e3b0bacde,7590cf109f6ff3277fd18d10f4727a8777d675ce,9c3712b517b7d0aa783ba97333fe953efd96e490,bf7564a66954eccc3ef6e43e4a288d9faf20f2b1,b9b5302a4c0a7261036a2d611d92aea3ae4d6a27,5a06410a6ec68fe858587a2069c73a2851a8c13c,1121a047a015aa03932d887f1d0e7627303513c6,c974a4819ba18a143903a6c66ffa0f40d5f547cd,c2fb01351b51091f68c750548c6e95acc9620552
2,1578463925048,173,203.69691,157.82071,B1,-1,5e15730ca280850006f3d007,5a0546857ecc773753327266,-0.578438,4.372969,8.584651,-0.241384,3.437459,9.053212,-0.799958,2.167658,9.468533,29.176820,3.650017,-28.407432,40.885102,31.467371,3.055417,-27.382297,41.824887,34.754372,2.327975,-24.383423,42.518669,0.183883,0.133984,0.492914,0.142394,0.113433,0.557229,0.092687,0.109112,0.610367,-0.160511,0.094585,0.212725,-0.149441,0.078972,0.185978,-0.046031,0.032992,0.097615,1896,-0.455077,0.079930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1578463926976,2101,203.69691,157.82071,B1,-1,5e15730ca280850006f3d007,5a0546857ecc773753327266,-0.283426,2.144523,10.032668,-0.737862,2.516724,9.300895,-0.796283,2.334164,9.444909,35.487088,1.192713,-25.055101,43.457037,34.509493,1.921536,-24.963078,42.635111,35.709524,-0.233099,-22.565005,42.242205,0.078985,0.097699,0.649827,0.101879,0.116730,0.605672,0.096510,0.124460,0.631369,-0.103517,0.050919,0.229180,-0.086032,0.054920,0.149323,-0.000182,0.039158,0.059482,32,-0.455077,0.079930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

100%|██████████| 3/3 [00:00<00:00, 5777.28it/s]


1,wifi_ts,wps_diff,x,y,floor,floor_int,file_id,site_id,acce_1000_x_avg,acce_1000_y_avg,acce_1000_z_avg,acce_2000_x_avg,acce_2000_y_avg,acce_2000_z_avg,acce_5000_x_avg,acce_5000_y_avg,acce_5000_z_avg,magn_1000_x_avg,magn_1000_y_avg,magn_1000_z_avg,magn_1000_st_avg,magn_2000_x_avg,magn_2000_y_avg,magn_2000_z_avg,magn_2000_st_avg,magn_5000_x_avg,magn_5000_y_avg,magn_5000_z_avg,magn_5000_st_avg,ahrs_1000_x_avg,ahrs_1000_y_avg,ahrs_1000_z_avg,ahrs_2000_x_avg,ahrs_2000_y_avg,ahrs_2000_z_avg,ahrs_5000_x_avg,ahrs_5000_y_avg,ahrs_5000_z_avg,gyro_1000_x_avg,gyro_1000_y_avg,gyro_1000_z_avg,gyro_2000_x_avg,gyro_2000_y_avg,gyro_2000_z_avg,gyro_5000_x_avg,gyro_5000_y_avg,gyro_5000_z_avg,rel_diff,rel_x,rel_y,...,43a3a0c7b91dfbd860ee7e0def592d095a91fd10,a6a1f9d7f56e54840d18ce91e08b78f803d4f244,98390bc208a2972ee38a65ab9a03875acf27c04f,ebc6647ca157db4e19b125cfe75edd373f931c2a,a72047c0e2918e08a4932c00442f4a60b5157723,ca04032e95b9f47cfad2335f641125ef290bec5f,e820af0e5672d463bd0d5913bb37102df4cd89bf,12a92794fe1f3dbee1ecbce0402f02ff21f74f07,ecae5bbc82f67a437d7654796e58eafd556c5817,c89c4c68eae3744d88708f01dc511cdc29f5469a,87a699fc6afc7378d3e40b391390e6b1080b8962,1973e49f645f93ac4a31f51b2f092f0c7af74f3f,d6e24e8be7dd37bf30f8d9762e5a17a7314ef688,23a37342da174eb7435274edd437df0f9c297fba,03d96d887bbf6de8fb4737e0aa2775397a782ab7,93ed3998aa51628a465185908cf2b1d59f64044e,015c7c64b6a54349e2397de357f568512772fb3e,0f4ec110e58259a5f7b543b46b7e17224b8c65d1,37c4029e22d476bcb51ce16a288128f1447c495e,6855f84c32177a43475d32f990736e32c4b9a68f,c24538de1bd60852828472078aa8a7370abf9b07,1d47f0a322dcde484b263509b8f901fe2b54126e,c46c048eaf0942a487310dfaf69e25e1c9d13481,9eab99f3f0c7c60a462019d91e74f1bf7d718140,a1ac46a6733a2d2653578cfccc5bac36de75ee87,d4f3fa21b68384dbf851e0709229738415c5565c,11705beec0b7e2eaffaba29308500e880ea7dafe,8de63284315d9ae45dc951091a8cf0b89d758d95,192396d26041e93147fe2f8b9d71074cd8a3d8c0,497b541a666e60032fb2fd36a7bc7f4a49aa5f3c,379d838fdf292e5976533a9f1372006b15745e06,81f80beb7d60e32595870f7bb8f84fa03fff51cb,1bbec7e1703b6d64655ea239c9ec5eb6cfb2f719,f38100fe6e17418c12e21f8cc53aff633f7c1904,5b65158e0c8c7db9cd426000a36270e0fd90b64f,a67a0c5bd3a7270e44d4ff5ef5b12807bff57d96,a683811bb6b9409bafaf886a3abc5db32e12cf0f,46f2f50e10aa181335ce41092846c867943b0308,8f202ce7446beb55b8974edc135c3b691c25dea8,c4e1a0e58223260a7f22907b91b3a1366f03ebf4,b5191bf39a4c9fd75908763280c621b67ba53abd,1b658216894683bfd2a2555bc4cf631782d22ca9,7c5b9115e2e9e2c6b8dc1c482a66d13d96b031a1,9ae6f0dbc76c4b460ddedf4906a7d02c7fd90498,8c9bc88e26dc86d5175c9e2d5ce16ff5da568084,226c392b2a7dd8b51e72d4ee9d72a80aeea4c4ed,db0513b9c7b8d54e3c70a381176b004e5158a879,fce721c3598ba996721a99b79c578272c3184344,6b46d02cb5332fa8faee62ef17ee4fbbf2fba372,566c5b793e21520537989a63145408ac0d19fa76
2,1560757675096,1973,292.59036,130.21555,B1,-1,5d0761f64cae4f000a2db584,5c3c44b80379370013e0fd2b,-1.156086,1.363079,9.612801,-1.236395,1.463620,9.671881,-1.376796,1.123597,9.650023,-22.931528,11.491877,-33.065388,41.847797,-21.921622,10.786970,-33.706798,41.630091,-21.377014,11.590374,-33.709941,41.565294,0.083465,0.015448,-0.507159,0.092206,0.015095,-0.499077,0.087749,0.025558,-0.500517,-0.019257,-0.009340,-0.060271,-0.022636,0.000427,-0.018270,-0.014086,-0.004410,-0.016048,309,0.354628,0.185038,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1560757677204,4081,292.59036,130.21555,B1,-1,5d0761f64cae4f000a2db584,5c3c44b80379370013e0fd2b,-1.581373,1.016716,9.872318,-1.552666,0.989999,9.604211,-1.393701,1.111349,9.697130,-19.374205,10.556589,-33.507107,40.118919,-20.408221,11.425295,-33.512043,40.866733,-20.649335,11.237525,-34.563859,41.801165,0.089028,0.034496,-0.496758,0.087179,0.034277,-0.502381,0.087209,0.026247,-0.500624,-0.024388,0.023256,0.034217,-0.007603,-0.013182,0.004523,-0.007925,-0.000205,-0.018236,296,0.503956,0.269088,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

100%|██████████| 3/3 [00:00<00:00, 3741.57it/s]


1,wifi_ts,wps_diff,x,y,floor,floor_int,file_id,site_id,acce_1000_x_avg,acce_1000_y_avg,acce_1000_z_avg,acce_2000_x_avg,acce_2000_y_avg,acce_2000_z_avg,acce_5000_x_avg,acce_5000_y_avg,acce_5000_z_avg,magn_1000_x_avg,magn_1000_y_avg,magn_1000_z_avg,magn_1000_st_avg,magn_2000_x_avg,magn_2000_y_avg,magn_2000_z_avg,magn_2000_st_avg,magn_5000_x_avg,magn_5000_y_avg,magn_5000_z_avg,magn_5000_st_avg,ahrs_1000_x_avg,ahrs_1000_y_avg,ahrs_1000_z_avg,ahrs_2000_x_avg,ahrs_2000_y_avg,ahrs_2000_z_avg,ahrs_5000_x_avg,ahrs_5000_y_avg,ahrs_5000_z_avg,gyro_1000_x_avg,gyro_1000_y_avg,gyro_1000_z_avg,gyro_2000_x_avg,gyro_2000_y_avg,gyro_2000_z_avg,gyro_5000_x_avg,gyro_5000_y_avg,gyro_5000_z_avg,rel_diff,rel_x,rel_y,...,c2bc7796f2a5e115111981a09ce62b19e1575695,049144e336266fdd2b0442a4fdd5edd883f2b3bb,cc37ffd6f06385eeaa726a07f2a72bf4cb65f2fe,12e31f0421188dd1ec74a9bc4f477c19515763f2,7f2356d411b6cd613766fbb2c6636d35b315d8aa,1963ebb4af96594e73702ab91e0197d24f498d54,a57c52fbf1cc3cd5c88b07c91c537e2d5e6d70ed,9e435bc304e3442e99932b339627d91285b72416,fbba48a1a65c328261c5571fb8d0b271c03f8f04,e9203b6e043cc5d0f8183a2acf7878077c35e802,3231ad7d29af36b8e40f6e2c69346fa7e757d860,238d3b5cb0a67eb87f29590c163d4a055cec049d,7a0b6966f159ba7c1d7a3e99adfd4cae713fdae4,fabb1338015ce1868604751fb204a8faea389c32,c720dbbdcda81838860b388f3f34c008bcaa5060,68da1421e1738cb556c9b33f53a881583cad4f16,059989c10c2583daeb02315e196a56a1b75f9d21,ab49f877da8c6ec07b3f8c88490aaa5aed279fe3,a4207575c7ad8f6dde09349c2f6ca286fb825cb8,8afb50a7b6e6404377a498a7c8df39d43afa812a,1db86b9a5f864cc5bd75709f87d5bd617785d538,ae3855bc2b1708a9f7fb174166c444e95a2a5d59,97e7b437b5fe361fba1946b5ed5e93612d194bce,e19abd63b589e1e647736afbcde911bf257bb360,d106a88d03756d49786e2f7f5e7126e43e32dd7e,adc38e7a478bb25e4e8f0cf1d59d569112851550,33178ce71454fbcaaf494de5437e7462e3284f27,94586be8ec683b2e96c563781dee83a55a6c9573,bc594a6bbea419a5b3b4efd6b48220c8117d6b87,7c7f8fabe86e524336441c00a2eb2fecde1a7d13,fee4e314dba20dec7c2348cd40082713afce1f88,5b554c0058be463688273b58f50e60c1c3551c91,89d02d04dc3a1fee60cfa8f8053a4751f5511804,f1aa98616b4a30c8a32af097a5d2d9495d6978ba,80c752437b5c186b7174802888890e071e2ac943,ac4ea4e30d154c299974075e8fcdcd827dde160a,384890dd1ec80815e858203243f39e5b1a1d0f31,32e722a57217252eeffe8d32a6697d0e007cc8ae,6771ff619310ac1b5b2b93fe056d1b973d73f308,0ebe0407c0f8da3cb8acc762c2f1c9775df8f3f3,b229021f01f7245190e9c6d08bfd72df18598eff,b320671870aa7c8649bc881360e1fdc2860ad2f7,62c31c0a3611ced1c239b69fd4f4a73fcfb137be,48d59d818652276967346830ddef54c64e6bcd77,7e7abe0862ec45ba4cd4f37c135fb719f117f924,0c768efecace42578ce5c7e5f4ea66cf570c45ff,5f37c5aa39dadf76a4555dd9a64809e871930d4e,990c46ee4118b03e0be593e2b07ef72488c82bdc,13affa23c8d8f9c3b18a7f65f438d55a0529b2c4,b65e1476942da139a8966cc4f6b3d8e5d60c3d45
2,1560137760438,662,73.254776,42.315132,F3,2,5cfdd04f6fa436000a02de4a,5cd56865eb294480de7167b6,-0.373220,2.818485,9.631410,-0.569155,2.811287,9.530579,-0.639413,2.932438,9.445192,7.491538,23.926923,-30.419230,39.420177,8.728837,22.617674,-31.020930,39.370673,9.411489,19.804042,-31.218085,38.148985,0.183088,-0.040334,0.192347,0.194933,-0.027792,0.204075,0.207547,-0.011536,0.218615,0.011073,0.074622,-0.019692,0.056440,0.051924,0.003917,0.019665,0.033824,0.004358,391,-0.187570,0.367498,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1560137761140,1364,73.254776,42.315132,F3,2,5cfdd04f6fa436000a02de4a,5cd56865eb294480de7167b6,-0.637605,2.620494,9.606365,-0.532575,2.891117,9.405067,-0.665930,2.986535,9.351678,8.812871,23.407723,-30.904752,39.757916,8.616363,21.546909,-31.292727,38.958257,9.665299,19.756277,-31.088138,38.081503,0.187834,-0.030965,0.207214,0.203780,-0.023966,0.207479,0.207675,-0.006592,0.217252,0.066863,0.029374,0.015861,0.048300,0.037679,0.006111,0.024224,0.018772,-0.004302,201,-0.243570,0.545755,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

100%|██████████| 3/3 [00:00<00:00, 663.90it/s]

time to extract data:  42.00778412818909


---
## Test generator
---

In [85]:
# Test specific methods
def extract_nearest_wifi(wifi_datas, timestamp):
    diff_list = []
    wifi_ts = wifi_datas[:, 0]
    for ts in wifi_ts:
        diff = abs(int(timestamp) - int(ts))
        diff_list.append(diff)
    min_value = min(diff_list)
    diff_indices = [i for i, x in enumerate(diff_list) if x == min_value]
    wifi_datas = wifi_datas[diff_indices]
    return wifi_datas

def extract_correct_ts(ibeacon_datas, timestamp):
    if ibeacon_datas.shape[0] == 0:
        print("no beacon")
        return np.nan
    else:
        diff_list = []
        ibeacon_ts = ibeacon_datas[:, 0]
        for ts in ibeacon_ts:
            diff = abs(int(timestamp) - int(ts))
            diff_list.append(diff)
        min_value = min(diff_list)
        diff_indices = [i for i, x in enumerate(diff_list) if x == min_value]
        ibeacon_datas = ibeacon_datas[diff_indices].flatten()
        ibeacon_last_ts = int(ibeacon_datas[-1])
        ibeacon_ts = int(ibeacon_datas[0])
        correct_ts = ibeacon_last_ts - (ibeacon_ts - int(timestamp))
        return correct_ts

def make_wifi_df_test(zipped_paths):
    site_id, file_id, timestamp, site_path_timestamp = zipped_paths
    file_path = "../input/indoor-location-navigation/test/" + file_id + ".txt"
    datas = read_data_file_ed(file_path)
    
    # Put IMU data in dict for later iteration
    imu_dict = {}
    imu_dict["acce"] = datas.acce
    imu_dict["magn"] = datas.magn
    imu_dict["ahrs"] = datas.ahrs
    imu_dict["gyro"] = datas.gyro
    # acce_uncali = datas.acce_uncali
    imu_dict["magn_uncali"] = datas.magn_uncali # Only use magn for uncalibrated data, as it seems more important in initial modeling result
    # gyro_uncali = datas.gyro_uncali
    
    # Leave the non-imu data as they are
    wifi_datas = datas.wifi
    ibeacon_datas = datas.ibeacon
    wps = datas.waypoint
    rel_positions = calc_rel_positions(imu_dict["acce"], imu_dict["ahrs"])
    
    # print("wifi unique ts len: ", len(set(wifi_datas[:, 0])))

    # Make wifi df with wifi_ts
    wifi_datas = extract_nearest_wifi(wifi_datas, timestamp)
    
    dfs = []
    df = pd.DataFrame(wifi_datas[:,[0,2,4]])
    for wifi_ts, g in df.groupby(0):
        g = g.drop_duplicates(subset=1)
        tmp = g.iloc[:,1:]
        feat = tmp.set_index(1).T
        feat["site_path_timestamp"] = site_path_timestamp
        correct_ts = extract_correct_ts(ibeacon_datas, timestamp) # get corrected timestamp using the last timestamp in ibeacon data
        feat["correct_wps_ts"] = correct_ts if correct_ts is not np.nan else np.nan
        # feat["timestamp"] = timestamp
        # feat["wifi_ts"] = wifi_ts
        feat["wifi_ts"] = correct_ts + (int(wifi_ts) - int(timestamp))

        # get closest wps
        feat["wps_diff"] = abs(int(wifi_ts) - int(timestamp))
        feat["x"] = np.nan
        feat["y"] = np.nan

        # get floor and other path data
        feat["floor"] = np.nan
        feat["floor_int"] = np.nan
        feat["file_id"] = file_id
        feat["site_id"] = site_id

        # Loop over IMU_CUTS
        for key, imu in imu_dict.items():
            for imu_cut in IMU_CUTS:
                imu_avgs = extract_imu_rep(imu, timestamp, imu_cut)
                feat[f"{key}_{imu_cut}_x_avg"] = imu_avgs[0]
                feat[f"{key}_{imu_cut}_y_avg"] = imu_avgs[1]
                feat[f"{key}_{imu_cut}_z_avg"] = imu_avgs[2]
                if key == "magn":
                    feat[f"{key}_{imu_cut}_st_avg"] = extract_one_magn_strength(imu_avgs)
        
        # get closest relative positions that was worked out with acce and ahrs data
        rel_pos = extract_nearest_wps(rel_positions, wifi_ts)
        feat["rel_diff"] = rel_pos[0]
        feat["rel_x"] = rel_pos[1][1]
        feat["rel_y"] = rel_pos[1][2]
        
        dfs.append(feat)
    
    return dfs

def make_test_df(zipped_path, site):
    dfs_all = pool.map(make_wifi_df_test, tqdm(zipped_path))
    dfs_unpack = [row for df in dfs_all for row in df]
    wifi_df = pd.concat(dfs_unpack)
    all_cols = wifi_df.columns
    cols = [col for col in all_cols if len(col) < 20]
    cols = reversed(cols)
    wifi_df = shift_columns(cols, wifi_df)
    wifi_df = wifi_df.fillna(-999)
    convert_dtypes(wifi_df, tqdm(INT_COLS), int)
    convert_dtypes(wifi_df, tqdm(CAT_COLS), "category")
    # print(wifi_df.iloc[:, :30].info())
    wifi_df.to_csv(f"./test/{site}_test.csv", index=False)
    del wifi_df

In [43]:
# Get submission file
sub_df = pd.read_csv("/kaggle/input/indoor-location-navigation/sample_submission.csv")
sub_df[["site_id", "file_id", "timestamp"]] = sub_df["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))
sub_df = sub_df.drop(columns=["floor", "x", "y"])
# sub_df_site_list = sub_df["site_id"].unique()

In [86]:
start = time.time()
num_cores = multiprocessing.cpu_count()
pool = Pool(num_cores)

# 100 records:  33.47870922088623 sec
# comment out to run all
# sub_df = sub_df.sample(n=100)
sub_df = sub_df.iloc[:9, :]
test_sites = sub_df["site_id"].unique()

# Run generator for each building
for site in test_sites:
    sub_df_filtered = sub_df[sub_df["site_id"] == site]
    site_file_zip = list(zip(sub_df_filtered["site_id"], \
                             sub_df_filtered["file_id"], \
                             sub_df_filtered["timestamp"], \
                             sub_df_filtered["site_path_timestamp"]))
    make_test_df(site_file_zip, site)

# display(wifi_df.head())

print("time to extract data: ", time.time() - start)
pool.close()

100%|██████████| 9/9 [00:00<00:00, 1876.55it/s]


1,site_path_timestamp,correct_wps_ts,wifi_ts,wps_diff,x,y,floor,floor_int,file_id,site_id,acce_1000_x_avg,acce_1000_y_avg,acce_1000_z_avg,acce_2000_x_avg,acce_2000_y_avg,acce_2000_z_avg,acce_5000_x_avg,acce_5000_y_avg,acce_5000_z_avg,magn_1000_x_avg,magn_1000_y_avg,magn_1000_z_avg,magn_1000_st_avg,magn_2000_x_avg,magn_2000_y_avg,magn_2000_z_avg,magn_2000_st_avg,magn_5000_x_avg,magn_5000_y_avg,magn_5000_z_avg,magn_5000_st_avg,ahrs_1000_x_avg,ahrs_1000_y_avg,ahrs_1000_z_avg,ahrs_2000_x_avg,ahrs_2000_y_avg,ahrs_2000_z_avg,ahrs_5000_x_avg,ahrs_5000_y_avg,ahrs_5000_z_avg,gyro_1000_x_avg,gyro_1000_y_avg,gyro_1000_z_avg,gyro_2000_x_avg,gyro_2000_y_avg,gyro_2000_z_avg,gyro_5000_x_avg,gyro_5000_y_avg,gyro_5000_z_avg,rel_diff,...,02d90a0bc95e068a8f75e00b3bd78579cb996c89,11b29720f56e05203439593fc1d09d068c370362,fa57df29b8ab97259c61687eb817a4902a310f96,568ec799c7fba15a6b161b08eed9de98fb3f8f50,bd4c5464a4cffed962e9b9ada76016b8002e886b,0c2998c5087c3ab1468f27f92723554295cfb69c,25ad89cae93a9b0ed486615d0b181f7e81e20a9b,94f55028b4f8bcf6b216f0db4160177efc2543c5,3c6695768483eb6b30d3e6f40f1bfde02e740e62,f8a80be7fc2a0a4aaa438b6366abfada7b566cd6,ee857f56bfb497efe85aa2a60e4ac85dcd207af2,84fe52504769b4ee72e0796b73daaa31c36cc8ee,87f46fabed4bd757563312b411ad6588cf3052ea,a1a77f543bd73cc2a3d53fece451692cbbd665e0,499cdcbaf798d669e117c2e3758641fc2e55ac3f,954aac3562feb83f061e72d80a6757dd9fd0814a,7476ce0f5573ab0ac2ed31989508c683ff6e42a7,ff115962b9756008e3bd8b6a969d53b1ac7a4616,3890f542ddf35f45e48bf1d9ec2e4e825b220fba,c996ca27909de1c8a008007adb8d43d261f282ba,5ee75f958530d980fa6c9a7ff53d9a0967975c00,d348cc807438ac5372135b7c6b1cc4fb6805d156,971ed0b2533e0773971b24096751f4470d6c12c0,799239299197dc3ca286142cb5f0b145f820c9f0,0d12cf08b8b0776697f1bb7bf6bffaea9f43c2ad,bf7564a66954eccc3ef6e43e4a288d9faf20f2b1,9271d7134a4b8a338a01f6b2b6e3568d8dd1d02f,0ef6f62af7d5940e8ac3e083128e1bf51ebd71cf,a3b99fa358735107a850939f2ca6efc54b861100,7e617ecd1eebc43306335348da7bb5317743343f,a0c56ac903a7e3bfcf28bdac110195ab8184f1ee,742b87a3f6a1d8576d83518d589bc7227c8ab3aa,fdc189e5a19850397f37201f4acc378cfddcf0d6,bbd4301665e78327e8e85d0bbb2173f5e1e3d45f,f7086e32e70d1bd474debe5553053dbb7a0c09b9,2f3c7bdb7cef7dbee88bd04571b0ed64dcf5f5c7,846bf4c9616e1a9dba29e67d754bdbfcc60ba813,0a741c9b887c6f7793cc6a3d4c8fa221de5988ca,4f1e1ecd5f7c4de40a70f00b7724ec6c8dbd85c5,e66c816a5c9aa65f0da1a2b5e9930be43de9b52f,4936f4b5f66fbfa3d2264ada20e8ea1ba06fbfbc,ac373687e24203f13e7bcf2b994316037d05f5a1,3e8f00d2264d44a94eaf17b4c3912a69755f3f7e,60486781563e45fd73ac9038bf87249e1ee1e106,0a23758a5622e59426bd94da6702cf3262acb80d,961ae5c1f60b834a2fb2712c306c7e40276a2e91,6953ea15c2f2d947e1affb04b64aaf9b38856d28,df9f77aab0ec4c21ecb2cd5c3067cf7581cab4d6,1aa43824bb504e2fcde734eb18421f4890cd2894,3fc85c6ed457ddae1069baed856526b64eafa406
2,5a0546857ecc773753327266_046cfa46be49fc1083481...,NaN,NaN,2331,NaN,NaN,NaN,NaN,046cfa46be49fc10834815c6,5a0546857ecc773753327266,-0.033225,4.443197,8.658677,-0.082463,4.418365,8.896522,-0.519580,3.749711,9.135446,26.840626,2.208085,-38.039814,46.608179,27.786534,0.240580,-37.524069,46.692667,27.663296,-1.165910,-36.973048,46.191163,0.236958,0.094365,0.430658,0.232403,0.112652,0.450876,0.184891,0.127650,0.492598,-0.088104,-0.054522,0.037350,-0.007936,-0.018674,0.019902,0.014635,0.008668,0.028694,66,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5a0546857ecc773753327266_046cfa46be49fc1083481...,NaN,NaN,491,NaN,NaN,NaN,NaN,046cfa46be49fc10834815c6,5a0546857ecc773753327266,-1.048206,3.798536,8.933929,-1.150104,3.873736,8.923185,-0.767980,3.853866,8.983102,23.539826,-0.836536,-28.946169,37.318949,16.910661,1.244701,-29.000700,33.594052,11.135098,3.084629,-32.823181,34.797508,0.173001,0.166249,0.435596,0.163333,0.144523,0.326229,0.180945,0.096724,0.248339,-0.125305,-0.375643,-0.616186,-0.011139,-0.200284,-0.339367,0.045131,-0.061754,-0.145687,129,..

100%|██████████| 3/3 [00:00<00:00, 573.49it/s]


1,site_path_timestamp,correct_wps_ts,wifi_ts,wps_diff,x,y,floor,floor_int,file_id,site_id,acce_1000_x_avg,acce_1000_y_avg,acce_1000_z_avg,acce_2000_x_avg,acce_2000_y_avg,acce_2000_z_avg,acce_5000_x_avg,acce_5000_y_avg,acce_5000_z_avg,magn_1000_x_avg,magn_1000_y_avg,magn_1000_z_avg,magn_1000_st_avg,magn_2000_x_avg,magn_2000_y_avg,magn_2000_z_avg,magn_2000_st_avg,magn_5000_x_avg,magn_5000_y_avg,magn_5000_z_avg,magn_5000_st_avg,ahrs_1000_x_avg,ahrs_1000_y_avg,ahrs_1000_z_avg,ahrs_2000_x_avg,ahrs_2000_y_avg,ahrs_2000_z_avg,ahrs_5000_x_avg,ahrs_5000_y_avg,ahrs_5000_z_avg,gyro_1000_x_avg,gyro_1000_y_avg,gyro_1000_z_avg,gyro_2000_x_avg,gyro_2000_y_avg,gyro_2000_z_avg,gyro_5000_x_avg,gyro_5000_y_avg,gyro_5000_z_avg,rel_diff,...,02d90a0bc95e068a8f75e00b3bd78579cb996c89,11b29720f56e05203439593fc1d09d068c370362,fa57df29b8ab97259c61687eb817a4902a310f96,568ec799c7fba15a6b161b08eed9de98fb3f8f50,bd4c5464a4cffed962e9b9ada76016b8002e886b,0c2998c5087c3ab1468f27f92723554295cfb69c,25ad89cae93a9b0ed486615d0b181f7e81e20a9b,94f55028b4f8bcf6b216f0db4160177efc2543c5,3c6695768483eb6b30d3e6f40f1bfde02e740e62,f8a80be7fc2a0a4aaa438b6366abfada7b566cd6,ee857f56bfb497efe85aa2a60e4ac85dcd207af2,84fe52504769b4ee72e0796b73daaa31c36cc8ee,87f46fabed4bd757563312b411ad6588cf3052ea,a1a77f543bd73cc2a3d53fece451692cbbd665e0,499cdcbaf798d669e117c2e3758641fc2e55ac3f,954aac3562feb83f061e72d80a6757dd9fd0814a,7476ce0f5573ab0ac2ed31989508c683ff6e42a7,ff115962b9756008e3bd8b6a969d53b1ac7a4616,3890f542ddf35f45e48bf1d9ec2e4e825b220fba,c996ca27909de1c8a008007adb8d43d261f282ba,5ee75f958530d980fa6c9a7ff53d9a0967975c00,d348cc807438ac5372135b7c6b1cc4fb6805d156,971ed0b2533e0773971b24096751f4470d6c12c0,799239299197dc3ca286142cb5f0b145f820c9f0,0d12cf08b8b0776697f1bb7bf6bffaea9f43c2ad,bf7564a66954eccc3ef6e43e4a288d9faf20f2b1,9271d7134a4b8a338a01f6b2b6e3568d8dd1d02f,0ef6f62af7d5940e8ac3e083128e1bf51ebd71cf,a3b99fa358735107a850939f2ca6efc54b861100,7e617ecd1eebc43306335348da7bb5317743343f,a0c56ac903a7e3bfcf28bdac110195ab8184f1ee,742b87a3f6a1d8576d83518d589bc7227c8ab3aa,fdc189e5a19850397f37201f4acc378cfddcf0d6,bbd4301665e78327e8e85d0bbb2173f5e1e3d45f,f7086e32e70d1bd474debe5553053dbb7a0c09b9,2f3c7bdb7cef7dbee88bd04571b0ed64dcf5f5c7,846bf4c9616e1a9dba29e67d754bdbfcc60ba813,0a741c9b887c6f7793cc6a3d4c8fa221de5988ca,4f1e1ecd5f7c4de40a70f00b7724ec6c8dbd85c5,e66c816a5c9aa65f0da1a2b5e9930be43de9b52f,4936f4b5f66fbfa3d2264ada20e8ea1ba06fbfbc,ac373687e24203f13e7bcf2b994316037d05f5a1,3e8f00d2264d44a94eaf17b4c3912a69755f3f7e,60486781563e45fd73ac9038bf87249e1ee1e106,0a23758a5622e59426bd94da6702cf3262acb80d,961ae5c1f60b834a2fb2712c306c7e40276a2e91,6953ea15c2f2d947e1affb04b64aaf9b38856d28,df9f77aab0ec4c21ecb2cd5c3067cf7581cab4d6,1aa43824bb504e2fcde734eb18421f4890cd2894,3fc85c6ed457ddae1069baed856526b64eafa406
2,5a0546857ecc773753327266_046cfa46be49fc1083481...,-999.0,-999,2331,-999.0,-999.0,-999.0,-999.0,046cfa46be49fc10834815c6,5a0546857ecc773753327266,-0.033225,4.443197,8.658677,-0.082463,4.418365,8.896522,-0.519580,3.749711,9.135446,26.840626,2.208085,-38.039814,46.608179,27.786534,0.240580,-37.524069,46.692667,27.663296,-1.165910,-36.973048,46.191163,0.236958,0.094365,0.430658,0.232403,0.112652,0.450876,0.184891,0.127650,0.492598,-0.088104,-0.054522,0.037350,-0.007936,-0.018674,0.019902,0.014635,0.008668,0.028694,66,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,5a0546857ecc773753327266_046cfa46be49fc1083481...,-999.0,-999,491,-999.0,-999.0,-999.0,-999.0,046cfa46be49fc10834815c6,5a0546857ecc773753327266,-1.048206,3.798536,8.933929,-1.150104,3.873736,8.923185,-0.767980,3.853866,8.983102,23.539826,-0.836536,-28.946169,37.318949,16.910661,1.244701,-29.000700,33.594052,11.135098,3.084629,-32.823181,34.797508,0.173001,0.166249,0.435596,0.163333,0.144523,0.326229,0.180945,0.096724,0.248339,-0.125305,-0.

FileNotFoundError: [Errno 2] No such file or directory: './test/5a0546857ecc773753327266_test.csv'

In [ ]:
# start = time.time()

# num_cores = multiprocessing.cpu_count()
# print(f"num_cores={num_cores}")
# pool = Pool(num_cores)

# # 10 paths:  6.070369720458984
# # 100 paths:  87.05400061607361
# # dfs_all = pool.map(make_wifi_df, tqdm(train_paths[:TRAIN_NUM]))
# dfs_all = pool.map(make_wifi_df, tqdm(grouped_paths_list[:10]))

# # time to process:  11.514546155929565
# # dfs_all = []
# # for path in train_paths[:TRAIN_NUM]:
# #     dfs_all.append(make_wifi_df(path))

# print(len(dfs_all))
# print("time to extract data: ", time.time() - start)
# pool.close()

In [ ]:
# start = time.time()

# num_cores = multiprocessing.cpu_count()
# pool = Pool(num_cores)

# # Do this for each building

# # 10 paths:  8.992910146713257
# # 100 paths:  2454.589078426361
# dfs_unpack = [row for df in dfs_all for row in df]
# wifi_df = pd.concat(dfs_unpack)

# print("time for df conversion: ", time.time() - start)
# print(len(wifi_df.columns))
# print(len(wifi_df))
# display(wifi_df.head())
# pool.close()

In [ ]:
# wifi_df.iloc[:,:50].info()

In [ ]:
# start = time.time()

# # move columns
# cols = ["acce_z_avg", "acce_y_avg", "acce_x_avg", \
#         "site_id", "file_id", "floor_int", "floor", \
#         "y", "x", "wps_diff", "wifi_ts"]

# for col in cols:
#     df_cols = list(wifi_df.columns)
#     df_cols.insert(0, df_cols.pop(df_cols.index(col)))
#     wifi_df = wifi_df[df_cols]
  
# # Fillna
# wifi_df = wifi_df.fillna(-999)

# display(wifi_df.head())
# print(len(wifi_df))

# print("time to shift columns: ", time.time() - start)
# print(wifi_df.iloc[:,:50].info())

In [ ]:
# print("available RAM:", psutil.virtual_memory())

# train_file_name = "indoor_train_5.pkl"

# with open(train_file_name, "wb") as file:
#     pickle.dump(wifi_df, file)

# del wifi_df
# del dfs_unpack
# del dfs_all
# gc.collect()

# print("available RAM after cleanup:", psutil.virtual_memory())

In [ ]:
# # Load data it back in
# train_file_name = "indoor_train_5.pkl"

# with open(train_file_name, "rb") as file:
#     df_train = pickle.load(file)

In [ ]:
# print("df len: ", len(df_train), "\n")
# print("site_id nunique: ", df_train["site_id"].nunique(), "\n")
# print("site_id value_counts: ", df_train["site_id"].value_counts(), "\n")
# print("file_id nunique: ", df_train["file_id"].nunique(), "\n")
# print("x value_counts: ", df_train["x"].value_counts(), "\n")
# print("y value_counts: ", df_train["y"].value_counts(), "\n")
# print("wifi_ts nunique: ", df_train["wifi_ts"].nunique(), "\n")
# print("wps_diff nunique: ", df_train["wps_diff"].nunique(), "\n")
# display(df_train.head())

In [ ]:
# df_train_pp = df_train.loc[:, ["site_id", "x", "y", "acce_x_avg", "acce_y_avg", "acce_z_avg"]]
# display(df_train_pp.head())
# sns.pairplot(df_train_pp, hue="site_id")

In [ ]:
# # Check the wps_diff distribution
# # Need to filter out those wps that are above 5000ms difference from wifi_ts
# f, ax = plt.subplots(figsize=(8, 8))
# f.patch.set_facecolor("white")
# sns.distplot(df_train["wps_diff"])
# plt.show()

In [ ]:
# df_train_slim = df_train[df_train["wps_diff"] < WPS_CUT]
# perc = round(len(df_train_slim)/len(df_train)*100, 2)

# print("no of records: ", len(df_train))
# print(f"Filter df_train with {WPS_CUT}, it retains {perc} % of data")

In [ ]:
# # Visualizing timestamp distribution

# # LabelEncode site_id, file_id, floor_converted, ssid, bssid
# # def col_encode(df, cols):
# #     for col in cols:
# #         le = preprocessing.LabelEncoder()
# #         df["%s_le"%col] = le.fit_transform(df[col])

# # col_enc = ["site_id", "file_id", "wifi_ssid", "wifi_bssid", "beacon_ssid"]
# # col_encode(df_train, tqdm(col_enc))

# # convert data types of certain columns
# def convert_dtypes(df, col_list, dtype):
#     for col in col_list:
#         df[col] = df[col].astype(dtype)

# convert_dtypes(df_train, tqdm(["wifi_ts"]), int)
# convert_dtypes(df_train, tqdm(["file_id", "site_id", "floor"]), "category")

# # Check
# display(df_train.head())

In [ ]:
# # Methods for preprocessing train data: Timestamp handling
# def find_diff_ts(ts, data):
#     data_ts = data[0]
#     diff_ts = int(data_ts) - int(ts)
#     return diff_ts

# def find_start_ts(path):
#     with open(path, 'r', encoding='utf-8') as file:
#         lines = file.readlines()

#     for line_data in lines:
#         line_data = line_data.strip()
#         m = re.search(r"(?<=startTime.)(.*)", line_data)
#         start_ts = m.groups(0)
#         if m:
#             return (start_ts[0])

# def find_smallest_diff(t, data):
#     if data.size == 0:
#         return np.array([])
#     else:
#         data_ts = data[:, [0]]
#         diff = []
#         for ts in data_ts:
#             diff.append(abs(int(t) - int(ts)))
#         closest_index = np.argmin(diff) # if multiple records have the same value..?
#         return data[closest_index]

In [ ]:
# # Method for preprocessing train data: splitting acce/ahrs/gyro/magn
# def split_axis(data, start_ts):
#     if data.size == 0:
#         # print("no axis data")
#         return [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
#     else:
#         data_ts = data[0]
#         diff_ts = int(data[0]) - int(start_ts)
#         x_axis = data[1]
#         y_axis = data[2]
#         z_axis = data[3]
#         try:
#             accuracy = data[4]
#         except IndexError:
#             accuracy = np.nan
#         return [data_ts, diff_ts, x_axis, y_axis, z_axis, accuracy]

# # Method for preprocessing train data: splitting wifi
# def split_wifi(data, start_ts):
#     if data.size == 0:
#         # print("no wifi data")
#         return [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
#     else:
#         data_ts = data[0]
#         diff_ts = int(data[0]) - int(start_ts)
#         ssid = data[1]
#         bssid = data[2]
#         rssi = data[3]
#         if len(data) > 5:
#             freq = data[4]
#             last_seen_ts = data[5]
#         else:
#             freq = np.nan
#             last_seen_ts = data[-1]
#         return [data_ts, diff_ts, ssid, bssid, rssi, freq, last_seen_ts]

# # Method for preprocessing train data: splitting ibeacon
# def split_beacon(data, start_ts):
#     if data.size == 0:
#         # print("no beacon data")
#         return [np.nan, np.nan, np.nan, np.nan]
#     else:
#         data_ts = data[0]
#         diff_ts = int(data[0]) - int(start_ts)
#         ssid = data[1]
#         rssi = data[2]
#         return [data_ts, diff_ts, ssid, rssi]

# # Method for preprocessing train data: calc rel pos
# def split_rel_pos(data, start_ts):
#     if data.size == 0:
#         # print("no rel_pos data")
#         return [np.nan, np.nan, np.nan, np.nan]
#     else:
#         data_ts = data[0]
#         diff_ts = int(data[0]) - int(start_ts)
#         x_axis = data[1]
#         y_axis = data[2]
#         return [data_ts, diff_ts, x_axis, y_axis]

In [ ]:
# # Extract path and other data
# def extract_path(path, floor_map):
#     # split path
#     try:
#         ex_path = f"{path}"
#         ex_paths = ex_path.split("/")
#         site_id = ex_paths[4]
#         floor = ex_paths[5]
#         f = floor_map[floor]
#         file_id = ex_paths[6].split(".")[0]
#         return [site_id, file_id, f, floor]
#     except:
#         print("extract_path error")

# # Definitely needs to be refactored
# def extract_data(path):
#     start_ts = find_start_ts(path)
#     path_datas = read_data_file(path)
#     acce = path_datas.acce
#     ahrs = path_datas.ahrs
#     magn = path_datas.magn
#     gyro = path_datas.gyro
#     acce_uncali = path_datas.acce_uncali
#     magn_uncali = path_datas.magn_uncali
#     gyro_uncali = path_datas.gyro_uncali
#     wifi = path_datas.wifi
#     wps = path_datas.waypoint
#     ibeacon = path_datas.ibeacon
#     rel_positions = calc_rel_positions(acce, ahrs)

#     # Changed from: just extracting wps time stamps -> take all acce uncalib timestamps
#     # ts = np.unique(wps[:, [0]])
#     if acce_uncali.any():
#         # print("acce_uncali")
#         ts = np.unique(acce_uncali[:, [0]]) # take uncalibrated access, as sometimes access has less data
#     elif acce.any():
#         # print("acce")
#         ts = np.unique(acce[:, [0]])
#     else:
#         print("no acce or acce_uncali")

#     # extract data for each timestamp of waypoints
#     res = []
#     for t in ts:
#         try:
#             wp_closest = find_smallest_diff(t, wps)
#             closest_wp_ts = wp_closest[0]
#             diff_ts_wp_ts = abs(int(t) - int(closest_wp_ts))
#             # time_stamp_cut = 2000, only the records within 2 sec of waypoint are kept
#             if diff_ts_wp_ts < time_stamp_cut:
#                 # flag to indicate how close the data point is to the wps
#                 # print("diff_ts_wp_ts", diff_ts_wp_ts)
#                 within_100ms = True if abs(diff_ts_wp_ts) <= 100 else False
#                 within_200ms = True if abs(diff_ts_wp_ts) <= 200 else False
#                 x = wp_closest[1]
#                 y = wp_closest[2]
#                 # print("x, y: ", x, y)
#                 diff_start_ts = int(t) - int(start_ts)
#                 diff_start_wp_ts = int(closest_wp_ts) - int(start_ts)
#                 # print("diff_start_ts, diff_start_wp_ts: ", diff_start_ts, diff_start_wp_ts)
#                 acce_closest = split_axis(find_smallest_diff(t, acce), start_ts)
#                 ahrs_closest = split_axis(find_smallest_diff(t, ahrs), start_ts)
#                 magn_closest = split_axis(find_smallest_diff(t, magn), start_ts)
#                 magn_closest.append(extract_one_magn_strength(magn_closest)) # append magnetic strength only for the magn data
#                 gyro_closest = split_axis(find_smallest_diff(t, gyro), start_ts)
#                 # print("acce: ", acce_closest)
#                 # print("ahrs: ", ahrs_closest)
#                 # print("magn: ", magn_closest)
#                 # print("gyro: ", gyro_closest)
#                 acce_u_closest = split_axis(find_smallest_diff(t, acce_uncali), start_ts)
#                 magn_u_closest = split_axis(find_smallest_diff(t, magn_uncali), start_ts)
#                 gyro_u_closest = split_axis(find_smallest_diff(t, gyro_uncali), start_ts)
#                 # print("acce_u_closest: ", acce_u_closest)
#                 # print("magn_u_closest: ", magn_u_closest)
#                 # print("gyro_u_closest: ", gyro_u_closest)
#                 wifi_closest = split_wifi(find_smallest_diff(t, wifi), start_ts)
#                 if len(ibeacon) > 0:
#                     beacon_closest = split_beacon(find_smallest_diff(t, ibeacon), start_ts)
#                 else:
#                     beacon_closest = [np.nan, np.nan, np.nan, np.nan]
#                 rel_pos = split_rel_pos(find_smallest_diff(t, rel_positions), start_ts)
#                 # print([t, x, y, int(closest_wp_ts), acce_closest, acce_u_closest])
#                 res.append([int(t), start_ts, diff_start_ts, x, y, int(closest_wp_ts), diff_start_wp_ts, diff_ts_wp_ts, within_100ms, within_200ms] + \
#                            acce_closest + ahrs_closest + magn_closest + gyro_closest + \
#                            acce_u_closest + magn_u_closest + gyro_u_closest + \
#                            wifi_closest + beacon_closest + rel_pos
#                           )
#             else:
#                 # print("no wp made it through timestamp cut")
#                 continue
#         except Exception as exc:
#             pass
#             # print("Error message: ", exc)
#             # print("extract_test_data error")
#     return res

In [ ]:
# # %%timeit

# # 5.55 ms ± 1.76 ms per loop
# path, site, floorNo, floor_plan_filename, \
# json_plan_filename, width_meter, height_meter = pick_example(len(train_paths), train_paths)

# def one_trace_to_rows(path, floor_map):
#     try:
#         path_info = extract_path(path, floor_map)
#         data = extract_data(path)
#         # rows = list(itertools.chain(path_info, *data))
#         rows = []
#         for d in data:
#             row = path_info + d
#             rows.append(row)
#             # print("row: ", row)
#         return rows
#     except:
#         print("one_trace_to_rows error at: ", path)

# # path -> train/5cd56bdbe2acfd2d33b663c0/L3/5dfc8108241c3600064049b9.txt
# # time w/ for loop with 1 train_path -> 11.6
# # time w/ itertools.chain for 1 train_path -> 11.8
# start = time.time()
# path_info = extract_path(path, floor_map)
# print("path: ", path_info)
# rows = one_trace_to_rows(path, floor_map)
# print("time to process one train_path", time.time() - start)
# #print("col count: ", len(rows[0]))
# print("rows: ", rows)

In [ ]:
# # Run row making function for all training paths
# # print(train_paths[:10])
# import time
# start = time.time()

# all_rows = []
# for train_path in train_paths[:10]:
#     rows = one_trace_to_rows(train_path, floor_map)
#     all_rows.extend(rows)

# one_trace_df = pd.DataFrame(all_rows)
# display(len(one_trace_df))

# # Data below are the time it took to create the old version of training data (only waypoints)
# # without Pool
# # 10 -> 1.64 sec
# # 100 -> 28.12 sec
# # 1000 -> 286.67 sec
# # to process training (~26,000 files) -> ~7500 sec (~2hours)
# print(time.time() - start)

# with Pool
# no need for wrapper with pool.starmap -> https://qiita.com/okiyuki99/items/a54797cb44eb4ae571f6

# Memo about Pool
# with Pool
# 10 -> 1.09 sec
# 100 -> 12.35 sec
# 1000 -> 113.87 sec
# to process training (~26,000 files) -> ~3000 sec (~50min)

In [ ]:
# # Check if we can make df

# # column names
# col_names = ["site_id", "file_id", "floor_converted", "floor", \
#              "ts", "start_ts", "diff_start_ts", "x", "y", \
#              "closest_wp_ts", "diff_start_wp_ts", "diff_ts_wp_ts", "within_100ms", "within_200ms", \
#              "acce_ts", "diff_acce_ts", "acce_x", "acce_y", "acce_z", "acce_acc", \
#              "ahrs_ts", "diff_ahrs_ts", "ahrs_x", "ahrs_y", "ahrs_z", "ahrs_acc", \
#              "magn_ts", "diff_magn_ts", "magn_x", "magn_y", "magn_z", "magn_acc", "magn_strength",\
#              "gyro_ts", "diff_gyro_ts", "gyro_x", "gyro_y", "gyro_z", "gyro_acc", \
#              "acce_u_ts", "diff_acce_u_ts", "acce_u_x", "acce_u_y", "acce_u_z", "acce_u_acc", \
#              "magn_u_ts", "diff_magn_u_ts", "magn_u_x", "magn_u_y", "magn_u_z", "magn_u_acc", \
#              "gyro_u_ts", "diff_gyro_u_ts", "gyro_u_x", "gyro_u_y", "gyro_u_z", "gyro_u_acc", \
#              "wifi_ts", "diff_wifi_ts", "wifi_ssid", "wifi_bssid", "wifi_rssi", "wifi_freq", "wifi_last_seen_ts", \
#              "beacon_ts", "diff_beacon_ts", "beacon_ssid", "beacon_rssi", \
#              "rel_ts", "diff_rel_ts", "rel_x", "rel_y"
#             ]

# print(len(col_names))

# df = pd.DataFrame(rows, columns=col_names)
# print("df len: ", len(df))
# print("site_id nunique: ", df["site_id"].nunique())
# print("file_id nunique: ", df["file_id"].nunique())
# print("x value_counts: ", df["x"].value_counts())
# print("y value_counts: ", df["y"].value_counts())
# print("event ts nunique: ", df["ts"].nunique())
# print("start ts nunique: ", df["start_ts"].nunique()) # should be one
# print("diff_ts_wp_ts value_counts: ", df["diff_ts_wp_ts"].value_counts())
# print("diff_ts_wp_ts nunique: ", df["diff_ts_wp_ts"].nunique())
# print("within_100ms value_counts: ", df["within_100ms"].value_counts())
# print("within_100ms nunique: ", df["within_100ms"].nunique())
# print("within_100ms count: ", df["within_100ms"].count())
# print("within_200ms value_counts: ", df["within_200ms"].value_counts())
# print("within_200ms nunique: ", df["within_200ms"].nunique())
# print("within_200ms count: ", df["within_200ms"].count())
# display(df.head())

In [ ]:
# # Set pool
# num_cores = multiprocessing.cpu_count()
# print(f"num_cores={num_cores}")
# # args = [(p, floor_map) for p in train_paths[:train_num]]
# args = [(p, floor_map) for p in grouped_paths_list]
# pool = Pool(num_cores)

# start = time.time()
# # w/ 250ms settings, 3 random samples from each site_id
# # 2 paths -> 18.7 sec
# # 10 paths -> 315 sec (df len is 1994)
# # 100 paths -> 708 sec (df len is 7183)
# # all ~ 600 paths -> 

# # errors
# # grouped_paths_list -> 100 paths -> site_id: 8 errors, 27 correct
# # grouped_paths_list -> 100 paths -> file_id: 23 errors, 77 correct

# # all in one go -> xxx sec
# # array_split -> 5891.8 sec

# # all in one go
# # res = pool.starmap(one_trace_to_rows, args)

# # split the args
# res = []
# for arg in tqdm(np.array_split(args, 50)):
#     res.extend(pool.starmap(one_trace_to_rows, arg))

In [ ]:
############################## KEEP THIS CELL FOR LATER REF ##############################

# Error in ~20% of the train paths -> caused by not having acces_uncali to create the event timestamps

# error files
# /5cd56b5ae2acfd2d33b58548/1F/5cf20b29718b08000848aa0a.txt
# /5cd56b5ae2acfd2d33b58548/2F/5cf214bbc852a70008c01607.txt
# /5cd56b5ae2acfd2d33b58548/2F/5cf214bda50dc300099d34cc.txt
# /5cd56b61e2acfd2d33b58d20/F2/5d085df529994a0008202661.txt
# /5cd56b61e2acfd2d33b58d20/F2/5d085dea4a2bd40008d47468.txt
# /5cd56b61e2acfd2d33b58d20/F4/5d086c44d85da00008644fce.txt
# /5cd56b5ae2acfd2d33b5854a/F3/5d078bab0e86b60008036348.txt
# /5cd56b5ae2acfd2d33b5854a/B1/5d073ba64a19c000086c559b.txt
# /5cd56b5ae2acfd2d33b5854a/F1/5d07603e4cae4f000a2db525.txt
# /5cd56b63e2acfd2d33b591c2/F2/5d0b0668912a980009fe91f2.txt
# /5cd56b63e2acfd2d33b591c2/F1/5d0afbfb2f8a26000805b9cb.txt
# /5cd56b63e2acfd2d33b591c2/F1/5d0afbf92f8a26000805b9c9.txt
# /5cd56b64e2acfd2d33b592b3/F2/5d0c9321c99c56000836df18.txt
# /5cd56b64e2acfd2d33b592b3/F3/5d0c9952ea565d0008e34e8b.txt
# /5cd56b64e2acfd2d33b592b3/F4/5d0c9d65ea565d0008e34ea2.txt
# /5cd56b5ae2acfd2d33b58549/5F/5d0613514a19c000086c432a.txt
# /5cd56b5ae2acfd2d33b58549/2F/5d11a6089c50c70008fe89bc.txt
# /5cd56b79e2acfd2d33b5b74e/F3/5d0b01522f8a26000805ba3e.txt
# /5cd56b79e2acfd2d33b5b74e/F3/5d0b015e2f8a26000805ba44.txt
# /5cd56b79e2acfd2d33b5b74e/F1/5d0af3452f8a26000805b830.txt
# /5cd56b6be2acfd2d33b59d1f/F1/5d08a1545125450008037d87.txt
# /5cd56b6be2acfd2d33b59d1f/F1/5d08a14e3f461f0008dac56c.txt
# /5cd56b6be2acfd2d33b59d1f/F3/5d0896415125450008037c76.txt

# base_path = "../input/indoor-location-navigation/train"
# error_files = [
#     "/5cd56b5ae2acfd2d33b58548/1F/5cf20b29718b08000848aa0a.txt",
#     "/5cd56b61e2acfd2d33b58d20/F2/5d085dea4a2bd40008d47468.txt",
#     "/5cd56b61e2acfd2d33b58d20/F4/5d086c44d85da00008644fce.txt",
#     "/5cd56b5ae2acfd2d33b5854a/F3/5d078bab0e86b60008036348.txt",
#     "/5cd56b63e2acfd2d33b591c2/F1/5d0afbfb2f8a26000805b9cb.txt",
#     "/5cd56b63e2acfd2d33b591c2/F1/5d0afbf92f8a26000805b9c9.txt",
#     "/5cd56b5ae2acfd2d33b58549/2F/5d11a6089c50c70008fe89bc.txt",
#     "/5cd56b79e2acfd2d33b5b74e/F3/5d0b01522f8a26000805ba3e.txt",
#     "/5cd56b6be2acfd2d33b59d1f/F1/5d08a1545125450008037d87.txt",
#     "/5cd56b6be2acfd2d33b59d1f/F1/5d08a14e3f461f0008dac56c.txt"
# ]

# working_path = "../input/indoor-location-navigation/train/5d2709c303f801723c3299ee/1F/5dad7d6daa1d300006faa80c.txt"
# error_paths = [base_path + e for e in error_files]
# rows = one_trace_to_rows(error_paths[1], floor_map)
# print(rows)

In [ ]:
# start = time.time()

# df_train = pd.DataFrame(res[0], columns=col_names)
# for r in res[1:]:
#     df = pd.DataFrame(r, columns=col_names)
#     df_train = df_train.append(df, ignore_index=True)

# print("time to process", time.time() - start)
# print("length of df made", len(df_train))
# display(df_train.head(10))

In [ ]:
# def list_to_df(row_list):
#     df_train = pd.DataFrame(row_list[0], columns=col_names)
#     for r in row_list[1:]:
#         df = pd.DataFrame(r, columns=col_names)
#         df_train = df_train.append(df)
#     return df_train

# start = time.time()
# pool = Pool(num_cores)

# df_train = pool.map(list_to_df, tqdm(res))

# # print("train_path count", len(train_paths[:train_num]))
# print("time to process", time.time() - start)
# print("length of df made", len(df_train))
# display(df_train.head(10))
# pool.close()

In [ ]:
# Calculate moving averages
# Differencing respect to time (as each timestep is unevenly spaced)

In [ ]:
# # Save the file in parquet
# # https://www.kaggle.com/pedrocouto39/fast-reading-w-pickle-feather-parquet-jay
# # https://www.kaggle.com/prmohanty/python-how-to-save-and-load-ml-models

# # Saving train data
# train_file_name = "indoor_train_4.pkl"

# with open(train_file_name, "wb") as file:
#     pickle.dump(df_train, file)

# # Save them to output
# # df_train.to_csv('df_train_2.csv',index=False)
# # df_test.to_csv('df_test.csv',index=False)

In [ ]:
# # Load data it back in
# with open(train_file_name, "rb") as file:
#     df_train = pickle.load(file)

In [ ]:
# print("df len: ", len(df_train), "\n")
# print("file_id unique: ", (df_train["file_id"].nunique()), "\n")
# print("site_id unique: ", (df_train["site_id"].nunique()), "\n")
# print("site_id value_counts: ", (df_train["site_id"].value_counts()))
# display(df_train.head())

In [ ]:
# # Get submission file
# sub_df = pd.read_csv("/kaggle/input/indoor-location-navigation/sample_submission.csv")
# sub_df[["site", "file", "timestamp"]] = sub_df["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))
# sub_df = sub_df.drop(columns=["floor", "x", "y"])
# # grouped_df = sub_df.groupby("file").sample(n=2)
# # all_file_id = grouped_df["file"].unique()
# # print(len(grouped_df))
# # print(len(all_file_id))
# # display(grouped_df.head())
# display(sub_df.head())

# test_site_id = sub_df["site"].unique()
# train_site_id = df_train["site_id"].unique()
# print(test_site_id, "\n")
# print(train_site_id, "\n")
# a = list(set(test_site_id) & set(train_site_id))
# print(a)